In [19]:
from finta import TA, utils
from data_provider import *
dm = DataModel()
dm.read_from_csvs("../../xcels", ["master0.csv", "master1.csv"])

In [ ]:
def calc_best_stochastic():
    from matplotlib import pyplot as plt
    from tqdm import tqdm, trange
    
    buys_lens = [0]
    buys = []
    sells_lens = [0]
    sells = []


    for symbol in tqdm(dm.TA_SYMBOLS):
        df = dm.get(symbol)
        buy_lens = []
        buy = []
        sell_lens = []
        sell = []
        
        for s in range(15, 30):
            for l in range(75, 90):
                def stoch_wrapper(x):
                    return stochastic_signal(x, s, l, 30)
                
                res = profit_in_interval(df, stoch_wrapper)
                buy.append([s, l, res[0][0].mean])
                buy_lens.append([s, l, res[0][1].mean])
                sell.append([s, l, res[1][0].mean])
                sell_lens.append([s, l, res[1][1].mean])
        buy = np.array(buy)
        sell = np.array(sell)
        buys.append(buy[np.argmax(buy, axis = 0)[2]])
        sells.append(sell[np.argmax(sell, axis = 0)[2]])

    print(sells)
    print(buys)


In [101]:
def calc_best_SMA()
    from matplotlib import pyplot as plt
    from tqdm import tqdm, trange
    pd.options.mode.chained_assignment = 'raise'

    def sma1545(x):
        return SMA_signal(x, 15, 50, 90)

    buys_lens = [0]
    buys = []
    sells_lens = [0]
    sells = []


    for symbol in tqdm(dm.TA_SYMBOLS):
        df = dm.get(symbol)
        buy_lens = []
        buy = []
        sell_lens = []
        sell = []
        for s in range(10, 20):
            for l in range(40, 50):
                def smasig(x):
                    return SMA_signal(x, s, l, 45)
                res = profit_in_interval(df, smasig)
                buy.append([s, l, res[0][0].mean])
                buy_lens.append([s, l, res[0][1].mean])
                sell.append([s, l, res[1][0].mean])
                sell_lens.append([s, l, res[1][1].mean])
        buy = np.array(buy)
        sell = np.array(sell)
        buys.append(buy[np.argmax(buy, axis = 0)[2]])
        sells.append(sell[np.argmax(sell, axis = 0)[2]])

    print(sells)
    print(buys)
    # print(f'overall: {stats.describe(buys)}')
    # print(f"overall:  {stats.describe(buys_lens)}")
    # print(f'overall:  {stats.describe(sells)}')
    # print(f'overall:  {stats.describe(sells_lens)}')

100%|██████████| 16/16 [01:49<00:00,  6.86s/it]

[array([19.        , 44.        , -0.06585082]), array([10.        , 42.        , -0.09947932]), array([15.        , 41.        , -0.08421309]), array([19.        , 49.        , -0.07745079]), array([ 1.70000000e+01,  4.00000000e+01, -3.90943459e-02]), array([17.        , 44.        , -0.08408884]), array([ 1.90000000e+01,  4.90000000e+01, -4.76009176e-02]), array([10.        , 41.        , -0.05689034]), array([19.        , 41.        , -0.08791469]), array([17.        , 49.        , -0.06743485]), array([ 1.40000000e+01,  4.90000000e+01, -4.74657324e-02]), array([10.        , 41.        , -0.07705426]), array([19.        , 44.        , -0.06585082]), array([10.        , 44.        , -0.06946047]), array([ 1.80000000e+01,  4.90000000e+01, -4.49615993e-02]), array([ 1.3000000e+01,  4.8000000e+01, -1.5707422e-02])]
[array([12.       , 48.       ,  0.2192493]), array([19.        , 47.        ,  0.20575757]), array([16.        , 49.        ,  0.26503322]), array([15.        , 41.        ,

In [91]:
import abc
from scipy import stats
from datetime import timedelta

class SignalerProfitAnalyser:
    class signal(abc.ABC):
        
        @abc.abstractmethod
        def __init__(self, future_length, *args, **kwargs):
            self.future_length = future_length
            self.args = args
            self.kwargs = kwargs

        @abc.abstractmethod
        def get_signals(self, symbol_data):
            pass
    
    def calculate_for_range(self, **kwargs):
        keys = kwargs.keys()
        lens = [len(kwargs[k]) for k in keys]
        mult = 1
        for l in lens:
            mult *= l
        for i in range(mult):
            calculate_for_range()
    
    def calculate_profits(self, symbol_data, signal_class):
        signals = signal_class.get_signals(symbol_data)
        buy_profits = []
        buy_profits_length = []
        sell_profits = []
        sell_profits_length = []
        for date, length, signal_type in signals:
            first_price = symbol_data.loc[date]["close"]
            buy = []
            sell = []
            df_close = symbol_data[date+timedelta(days=1):date+timedelta(days=length)]["close"]
            if len(df_close) > 0:
                if signal_type == "buy":
                    buy_profits.append((df_close.max() - first_price) / first_price)
                    buy_profits_length.append((df_close.idxmax() - date).days)
                elif signal_type == "sell":
                    sell_profits.append((df_close.min() - first_price) / first_price)
                    sell_profits_length.append((df_close.idxmin() - date).days)

        if len(buy_profits) == 0:
            buy_profits = [0, 0]
        if len(buy_profits_length) == 0:
            buy_profits_length = [0, 0]
        if len(sell_profits) == 0:
            sell_profits = [0, 0]
        if len(sell_profits_length) == 0:
            sell_profits_length = [0, 0]

        buy_profits = np.array(buy_profits)
        buy_profits_length = np.array(buy_profits_length)
        sell_profits = np.array(sell_profits)
        sell_profits_length = np.array(sell_profits_length)

        ans = [(stats.describe(buy_profits), stats.describe(buy_profits_length)),
               (stats.describe(sell_profits), stats.describe(sell_profits_length))]
        return ans


    class minus_signal(signal):
        def __init__(self, future_length, day_length):
            self.future_length = future_length
            self.day_length = day_length
        
        def get_signals(self, symbol_data):
            signals = []
            count = 0
            for date in symbol_data.index:
                if symbol_data.loc[date, "ending-percent"] < -2:
                    count = count + 1
                else:
                    if symbol_data.loc[date, "ending-percent"] > 0:
                        if count >= self.day_length:
                            signals.append((date, self.future_length, "buy"))
                    count = 0
            return signals

    class SMA_signal:
        
        def __init__(self, short_period, long_period, future_length):
            self.short_period = short_period
            self.long_period = long_period
            self.future_length = future_length
        
        def calcSma(data, smaPeriod):
            j = next(i for i, x in enumerate(data) if x is not None)
            our_range = range(len(data))[j + smaPeriod - 1:]
            empty_list = [None] * (j + smaPeriod - 1)
            sub_result = [np.mean(data[i - smaPeriod + 1: i + 1]) for i in our_range]
            return np.array(empty_list + sub_result)
        
        def crossover(series1, series2):
            return pd.Series(np.where(series1 > series2, 1.0, 0.0)).diff()
        
        def ema(data, days):
            return data.ewm(span=days).mean()

        def sma(data, days):
            return data.rolling(window=days).mean()
        
        def get_signals(self, symbol_data):
            signals = []
            close = pd.Series(symbol_data["close"])
            ssma = ema(close, self.short_period)
            lsma = ema(close, self.long_period)

            cross = crossover(ssma, lsma)
            cross.index = symbol_data.index
            buy = list(cross[cross > 0.0].index)
            sell = list(cross[cross < 0.0].index)
            for b in buy:
                signals.append((b, self.future_length, "buy"))
            for s in sell:
                signals.append((s, self.future_length, "sell"))
            return signals

    class stochastic_signal:
        
        def __init__(self, oversold, overbought, future_length):
            self.oversold = oversold
            self.overbought = overbought
            self.future_length = future_length
        
        def crossover(self, series1, series2):
            return pd.Series(np.where(series1 > series2, 1.0, 0.0)).diff()
        
        def get_signals(self, symbol_data):
            signals = []
            
            symbol_data["high"] = symbol_data["max"]
            symbol_data["low"] = symbol_data["min"]
            d=TA.STOCHD(df)
            k=TA.STOCH(df)
            cross = self.crossover(k, d)
            cross.index = symbol_data.index

            buy = list(cross[np.logical_and(cross > 0.0, d < self.oversold)].index)
            sell = list(cross[np.logical_and(cross < 0.0, d > self.overbought)].index)

            for b in buy:
                signals.append((b, self.future_length, "buy"))
            for s in sell:
                signals.append((s, self.future_length, "sell"))
            return signals


In [92]:
spa = SignalerProfitAnalyser()
df = dm.get(dm.TA_SYMBOLS[1])
def stoch_wrapper(x):
    return stochastic_signal(x, 20, 80, 30)

In [93]:
# np.seterr(all='raise')
# df.columns
# spa.profit_in_interval(df, spa.minus_signal(future_length=30, day_length=3))
spa.profit_in_interval(df, spa.stochastic_signal(future_length=30, oversold=80, overbought=20))

[(DescribeResult(nobs=470, minmax=(-0.0967741935483871, 1.0632661996497372), mean=0.09487142411347835, variance=0.020652722585960633, skewness=2.8287418512960354, kurtosis=10.516578033403285),
  DescribeResult(nobs=470, minmax=(1, 30), mean=13.61276595744681, variance=107.9606042734655, skewness=0.2707796506154837, kurtosis=-1.4226527816959995)),
 (DescribeResult(nobs=422, minmax=(-0.525, 0.12272727272727273), mean=-0.06555968742231755, variance=0.006302002887533689, skewness=-1.5518162731531608, kurtosis=4.955781135847541),
  DescribeResult(nobs=422, minmax=(1, 30), mean=15.364928909952607, variance=109.61947968614562, skewness=0.02867580732537232, kurtosis=-1.4972458940945765))]